In [ ]:
from google.colab import drive
drive.mount('/content/drive')

filepath = "/content/drive/MyDrive/web-BerkStan.mtx"

MessageError: Error: credential propagation was unsuccessful

**Section 1**

کد پایین مربوط به استفاده از روش مقدار ویژه هست.


In [ ]:
import numpy as np
import networkx as nx
from scipy.io import mmread

def load_berkeley_graph(filepath, num_nodes=1000):

    A = mmread(filepath).tocsr()

    G = nx.DiGraph(A)

    selected_nodes = sorted(G.nodes)[:num_nodes]

    G = G.subgraph(selected_nodes).copy()

    return G

def normalize_transition_matrix(A, alpha=0.85):

    num_nodes = A.shape[0]

    col_sums = A.sum(axis=0)

    dangling_nodes = (col_sums == 0)

    if np.any(dangling_nodes):

        A[:, dangling_nodes] = 1 / num_nodes

        col_sums = A.sum(axis=0)


    A = A / col_sums


    teleport = np.ones((num_nodes, num_nodes)) / num_nodes

    A = alpha * A + (1 - alpha) * teleport

    return A

def compute_pagerank_eigen(G, alpha=0.85):

    A = nx.to_numpy_array(G, dtype=float)

    A = normalize_transition_matrix(A, alpha)

    eigvals, eigvecs = np.linalg.eig(A)

    idx = np.argmax(np.abs(eigvals))

    r = np.real(eigvecs[:, idx])

    r = np.abs(r)
    return r / r.sum()

def main(filepath):

    G = load_berkeley_graph(filepath, num_nodes=1000)

    pagerank = compute_pagerank_eigen(G)

    top_20_ids = np.argsort(-pagerank)[:20]

    print("Top 20 IDs:", top_20_ids)


filepath = "/content/drive/MyDrive/web-BerkStan.mtx"
main(filepath)

Top 20 IDs: [509 702 612 692 371 459 564 425 256 860 717 295 329 197 699 741 232 160
 812 909]


کد پایین با استفاده از روش توانی می باشد.

In [ ]:
import numpy as np
import networkx as nx
from scipy.io import mmread

def load_berkeley_graph(filepath, num_nodes=1000):

    A = mmread(filepath).tocsr()

    G = nx.DiGraph(A)

    selected_nodes = sorted(G.nodes)[:num_nodes]

    G = G.subgraph(selected_nodes).copy()

    return G

def power_iteration(A, alpha=0.85, num_iter=1000, tol=1e-6):

    n = A.shape[0]

    x = np.ones(n) / n

    for _ in range(num_iter):

        x_new = alpha * (A @ x) + (1 - alpha) / n

        x_new /= np.sum(x_new)

        if np.linalg.norm(x_new - x, 1) < tol:
            break
        x = x_new

    return x

def compute_pagerank_power(G, alpha=0.85):

    A = nx.to_numpy_array(G, dtype=float)

    num_nodes = A.shape[0]


    col_sums = A.sum(axis=0)

    dangling_nodes = (col_sums == 0)


    if np.any(dangling_nodes):

        A[:, dangling_nodes] = 1 / num_nodes

        col_sums = A.sum(axis=0)

    col_sums[col_sums == 0] = 1

    A /= col_sums


    return power_iteration(A, alpha)

def main(filepath):

    G = load_berkeley_graph(filepath, num_nodes=1000)

    pagerank_power = compute_pagerank_power(G)

    top_20_indices = np.argsort(-pagerank_power)[:20]

    print("Top 20 Node IDs:", top_20_indices)


filepath = "/content/drive/MyDrive/web-BerkStan.mtx"

main(filepath)

Top 20 Node IDs: [509 702 612 692 371 459 564 425 256 860 717 295 329 197 699 741 232 160
 812 909]


با استفاده از پکیج networkx رتبه بندی میکنیم.

In [ ]:
import networkx as nx

def load_graph_from_file(filepath, num_nodes=1000):

    G = nx.DiGraph()

    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith('%'):
                continue

            parts = line.split()
            if len(parts) >= 2:
                i, j = map(int, parts[:2])
                G.add_edge(i, j)


    selected_nodes = sorted(G.nodes)[:num_nodes]
    G = G.subgraph(selected_nodes).copy()

    return G

def compute_pagerank_networkx(G, alpha=0.85):


    pagerank = nx.pagerank(G, alpha=alpha)
    return pagerank

def main(filepath):


    G = load_graph_from_file(filepath, num_nodes=1000)


    pagerank = compute_pagerank_networkx(G)


    top_20_nodes = sorted(pagerank, key=pagerank.get, reverse=True)[:20]


    print("Top 20 IDs with Highest PageRank:")
    for node in top_20_nodes:
        print(node)


filepath = "/content/drive/MyDrive/web-BerkStan.mtx"


main(filepath)

Top 20 IDs with Highest PageRank:
861
768
40
927
372
743
935
257
804
33
968
945
715
720
924
721
296
330
813
198
